In [1]:

import requests
from bs4 import BeautifulSoup
import re 
from datetime import datetime, timedelta
from requests.exceptions import ConnectionError
import time


In [2]:
cookie = 'Bifrost_SessionID=q0svamuliir35pmr0xag1c5t'

In [3]:
def run_get__id_list(cookie):
    def request_first_page(fromdate,todate):
        # URL yêu cầu
        url = f'https://ecosys.gov.vn/CertificatesUpgrade/Business/OrderView.aspx?CertificateDateFrom={fromdate}&CertificateDateTo={todate}&status=-1&CertificateNumber=&orderCode='

        # Đặt headers
        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            'Cookie': cookie,
            'DNT': '1',
            'Referer': 'https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx?',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"'
        }

        # Gửi yêu cầu GET
        response = requests.get(url, headers=headers,timeout=600)

        if response.status_code == 200:
            print(f"Request {fromdate},{todate} thành công!")
            return response.text
        # print(response2.text)  # Nội dung HTML
        else:
            print(f"Request thất bại: {response.status_code}")
            return None

    def request_more_page(eventvalidation,viewstate,page_index): 
        import requests

        # URL cần truy vấn
        more_page_url = "https://ecosys.gov.vn/CertificatesUpgrade/Business/OrderView.aspx"

        # Các headers sử dụng trong request
        headers2 = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Language": "en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7",
            "Cache-Control": "max-age=0",
            "Connection": "keep-alive",
            "Cookie": cookie,
            "DNT": "1",
            "Origin": "https://ecosys.gov.vn",
            "Referer": "https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx?CertificateDateFrom=01/01/2024&CertificateDateTo=31/03/2024&Status=-1&FormCOId=0&CountryId=0&CertificateNumber=&CustomsNumber=&ReceiverName=&TransportMethodId=0",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "same-origin",
            "Sec-Fetch-User": "?1",
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
            "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"Windows"',
        }

        # Dữ liệu form gửi kèm
        data2 = {
            
            "__EVENTVALIDATION": eventvalidation,
            "__VIEWSTATE":viewstate,
            "ctl00$cplhContainer$ddlChoiceIndexOfPage":page_index
        }

        # Gửi POST request
        response2 = requests.post(more_page_url, headers=headers2, data=data2,timeout=600)

        # Kiểm tra kết quả trả về
        if response2.status_code == 200:
            print(f"Request page {page_index} thành công!")
            return response2.text
            # print(response2.text)  # Nội dung HTML
        else:
            print(f"Request thất bại: {response2.status_code}")
            return None

    def generate_dates(start_date, end_date):
        """Tạo danh sách các khoảng thời gian cách nhau 1 tuần, lùi về đến ngày 01/01/2022."""
        periods = []
        current_start = datetime.strptime(start_date, "%d/%m/%Y")
        current_end = datetime.strptime(end_date, "%d/%m/%Y")

        while current_start >= datetime.strptime("01/01/2022", "%d/%m/%Y"):
            periods.append((current_start.strftime("%d/%m/%Y"), current_end.strftime("%d/%m/%Y")))
            # Lùi lại một tuần
            current_end = current_start - timedelta(days=1)
            current_start = current_end - timedelta(days=29)
        periods.append(("01/01/2022", current_end.strftime("%d/%m/%Y")))

        return periods

    # Ngày bắt đầu là ngày hiện tại
    start_date = (datetime.now() - timedelta(days=28)).strftime("%d/%m/%Y")
    # Ngày kết thúc là ngày hiện tại + 1 ngày
    end_date = (datetime.now() + timedelta(days=1)).strftime("%d/%m/%Y")

    # Tạo danh sách các khoảng thời gian
    dates_list = generate_dates(start_date, end_date)

    len(dates_list)


    id_pattern = r"\b\d{5,7}\b"
    # Danh sách lưu các ID tìm được
    ids = []

    # Số lần thử lại tối đa
    MAX_RETRIES = 10
    RETRY_DELAY = 5  # Giây

    for start, end in dates_list:
        retries = 0
        success = False
        
        while retries < MAX_RETRIES and not success:
            try:
                request_text = request_first_page(start, end)
                if request_text is not None:
                    soup = BeautifulSoup(request_text, "html.parser")
                    
                    viewstate = soup.find("input", attrs={"type": "hidden", "name": "__VIEWSTATE", "id": "__VIEWSTATE"})["value"]
                    eventvalidation = soup.find("input", attrs={"type": "hidden", "name": "__EVENTVALIDATION", "id": "__EVENTVALIDATION"})["value"]
                    total_page = int(soup.find("span", attrs={"id": "ctl00_cplhContainer_lblTotalPage"}).string)
                    
                    if not viewstate or not eventvalidation or not total_page:
                        raise ValueError(f"Missing data for request from {start} to {end}. Stopping.")
                    
                    # Xử lý các ID trên trang đầu tiên
                    all_td = soup.find_all("td", style="display:none;")
                    for td in all_td:
                        if re.fullmatch(id_pattern, td.text.strip()):
                            print(td.text.strip())
                            ids.append(td.text.strip())
                    
                    # Xử lý các trang tiếp theo (nếu có)
                    if total_page > 1:
                        for page_index in range(2, total_page + 1):
                            retries_inner = 0
                            while retries_inner < MAX_RETRIES:
                                try:
                                    request_text_2 = request_more_page(eventvalidation, viewstate, page_index)
                                    if request_text_2:
                                        soup2 = BeautifulSoup(request_text_2, "html.parser")
                                        all_td2 = soup2.find_all("td", style="display:none;")
                                        for td2 in all_td2:
                                            if re.fullmatch(id_pattern, td2.text.strip()):
                                                print(td2.text.strip())
                                                ids.append(td2.text.strip())
                                    break  # Thoát vòng lặp retry nếu thành công
                                except Exception as e:
                                    retries_inner += 1
                                    print(f"Error processing page {page_index} for {start} to {end}. Retry {retries_inner}/{MAX_RETRIES}. Error: {e}")
                                    time.sleep(RETRY_DELAY)
                    success = True  # Đánh dấu xử lý thành công
                else:
                    print(f"Request failed for {start} to {end}. Retrying...")
                    retries += 1
                    time.sleep(RETRY_DELAY)
            except ConnectionError as e:
                retries += 1
                print(f"Connection error for {start} to {end}. Retry {retries}/{MAX_RETRIES}. Error: {e}")
                time.sleep(RETRY_DELAY)
            except Exception as e:
                print(f"Unexpected error for {start} to {end}: {e}")
                break  # Ngừng thử nếu gặp lỗi không mong muốn

        if not success:
            print(f"Failed to process request for {start} to {end} after {MAX_RETRIES} retries.")

    len(ids)
    with open("output_invoice_ids.txt", "w", encoding="utf-8") as file:
        for item in ids:
            file.write(f"{item}\n")  
            
    with open("output_invoice_ids.txt", "r", encoding="utf-8") as file:
        invoice_list = [line.strip() for line in file]  
    

In [4]:
run_get__id_list(cookie)

Request 03/12/2024,01/01/2025 thành công!
Unexpected error for 03/12/2024 to 01/01/2025: 'NoneType' object has no attribute 'string'
Failed to process request for 03/12/2024 to 01/01/2025 after 10 retries.
Request 03/11/2024,02/12/2024 thành công!
1070842
Request 04/10/2024,02/11/2024 thành công!
1026167
977064
Request 04/09/2024,03/10/2024 thành công!
944288
919591
890255
Request 05/08/2024,03/09/2024 thành công!
841355
Request 06/07/2024,04/08/2024 thành công!
807225
767857
Request 06/06/2024,05/07/2024 thành công!
712135
673922
Request 07/05/2024,05/06/2024 thành công!
639677
Request 07/04/2024,06/05/2024 thành công!
591778
566714
Request 08/03/2024,06/04/2024 thành công!
520371
476221
Request 07/02/2024,07/03/2024 thành công!
450689
Request 08/01/2024,06/02/2024 thành công!
405584
370955
Request 09/12/2023,07/01/2024 thành công!
312068
292173
Request 09/11/2023,08/12/2023 thành công!
251946
Request 10/10/2023,08/11/2023 thành công!
204355
177901
Request 10/09/2023,09/10/2023 thành 